##### 2020 CAU 보안시스템 운영과 활용 
###### 네트워크 보안시스템 실습 프로젝트
20161856 한지효
<br>
<br>
## < 백신 만들어보기 >
<br>


### 1. 전용 백신
* <b>전용백신</b>: 하나의 악성코드를 진단 및 치료하는 백신  
* 해당 악성코드에 대해서는 빠르고 정확하지만 범욤성이 떨어진다.   


#### - 1.1 악성코드의 문자열(패턴)을 읽어 악성코드 여부 확인  

In [31]:
import os

file = './malwares/eicar1.txt'
fp = open(file, 'r')
fbuf = fp.read() # 파일 객체로부터 내용 읽어들여 버퍼에 저장
fp.close()

if fbuf[0:3] == 'X5O':    # 버퍼 내용의 첫 세 글자가 X5O일 경우 악성코드로 진단
    print('악성코드 발견!')
    os.chmod(file, 0o777) # 파일에 대한 권한 할당
    os.remove(file)       # 파일 강제 삭제 (치료)
    print('파일이 삭제되었습니다.')
else: print('악성코드가 없음')

악성코드 발견!
파일이 삭제되었습니다.


#### - 1.2 해시값 탐지 도입
<br>  
* 문자열만 탐지하기에는 오진 가능성 높음
* 강력하진 않아도 속도가 빠른 <b>MD5</b> 해시 알고리즘 이용  
<br>  
MD5 해시로 진단 가능 악성코드:  
바이러스 유형을 제외한 파일 그 자체가 악성코드인 것
###### ex) 트로이목마, 백도어, 웜

In [32]:
import hashlib # MD5를 구하기 위해 import
import os

file = './malwares/eicar2.txt'
fp = open(file, 'rb') # 바이너리 모드로 읽어들여 파일객체 생성
fbuf = fp.read()      # 파일객체로부터 내용 읽어들여 버퍼에 저장
fp.close()

f = hashlib.md5() # MD5 hash function
f.update(fbuf)    # hashing
hashValue = f.hexdigest() # 메시지 다이제스트를 얻음(16진수 해시값)

print(hashValue)

44d88612fea8a8f36de82e1278abb02f


In [33]:
# 악성코드의 해시값을 이용하여 진단 및 치료
if hashValue == '44d88612fea8a8f36de82e1278abb02f': # eicar2.txt 파일의 MD5 해시값
    print( '악성코드 발견!')
    os.chmod(file, 0o777)
    os.remove(file)
    print('파일이 삭제되었습니다.')
else: print('악성코드가 없음')

악성코드 발견!
파일이 삭제되었습니다.


### 2. 통합 백신
* DB에 등록된 여러 악성코드를 탐지

#### - 2.1 악성코드의 해시값이 담긴 DB와 검색 기능 생성

In [34]:
def makeDB():
    malwareDB = { # 파일 이름: 해시값, 파일 크기
        'EICAR' : ['44d88612fea8a8f36de82e1278abb02f', 68],
        'JIHYO' : ['4550049fcce72c47413b48c138ce72c4', 137]
    }
    return malwareDB

def searchDB(hashValue, malwareDB):
    for key, value in malwareDB.items(): # DB에 해당 해시값이 존재할 경우
        if value[0] == hashValue:
            return True, key             # 악성코드 이름과 함께 return
    return False, ''

#### - 2.2 악성코드를 진단하고 치료하는 백신 생성

In [35]:
def vaccine(fileLocation):
    malwareDB = makeDB() # DB 가져오기
    sizeDB = map(lambda value: value[1], malwareDB.values()) # 악성코드 크기만

    fp = open(fileLocation, 'rb')
    
    # ------------ 속도 향상을 위한 파일 크기 색출 ------------ #
    fileSize = os.path.getsize(fileLocation)
    if fileSize not in sizeDB:
        print(f'-- Normal File \n{fileLocation}')
        return # 악성코드에 해당하는 파일크기가 아니라면 pass
    # --------------------------------------------------------- #
    
    fbuf = fp.read()
    fp.close()

    # ---------------------- 해시값 탐지 ---------------------- #
    f = hashlib.md5() # MD5 hash function
    f.update(fbuf)    # hashing
    hashValue = f.hexdigest() # 메시지 다이제스트 get (16진수 해시값)
    
    # 구한 해시값을 넣어 악성코드 여부와 그 이름 탐지
    isMalware, name = searchDB(hashValue, malwareDB) 
    # --------------------------------------------------------- #
    
    
    if isMalware == True:
        print(f'-- 악성코드 탐지( {name} ) \n{fileLocation}')
        os.chmod(fileLocation, 0o777)
        os.remove(fileLocation)
        print('-- 파일이 삭제되었습니다.\n')
    else: print(f'-- Normal File \n{fileLocation}')

In [36]:
vaccine('./malwares/eicar3.txt')
vaccine('./malwares/jihyo1.txt')
vaccine('./malwares/normal.txt')

-- 악성코드 탐지( EICAR ) 
./malwares/eicar3.txt
-- 파일이 삭제되었습니다.

-- 악성코드 탐지( JIHYO ) 
./malwares/jihyo1.txt
-- 파일이 삭제되었습니다.

-- Normal File 
./malwares/normal.txt


### 3. 악성코드 정보가 담긴 (암호화된) DB파일을 통한 진단
* 악성코드 정보 진단의 용이를 위해 그 패턴(정보)만 따로 관리하는 DB 파일 생성
* 이때, 백신의 신뢰성(공격 및 오진 방지)을 위해 해당 파일을 암호화

#### - 3.1 AES 암호화 기능 도입
** 단순화를 위해 헤더 추가, 해시화 등은 제외함

In [37]:
import pyaes # import AES library
  
def AES_encrypt(key, plaintext):
    aes = pyaes.AESModeOfOperationCTR(str.encode(key)) # CTR mode chosen for stream cipher
    encryptedtext = aes.encrypt(plaintext) # cipher text creation
    return encryptedtext

def AES_decrypt(key, encryptedtext):
    aes = pyaes.AESModeOfOperationCTR(str.encode(key))
    decryptedtext = aes.decrypt(encryptedtext)    
    return decryptedtext

In [39]:
# 256 bit (32 byte) key chosen
key = '2020_CAU__Han_Jihyo_20161856_CSE'

p = input('Enter: ')
e = AES_encrypt(key, p)
print("\n The Encrypted text (in bytes): \n", e)
d = AES_decrypt(key, e)
print("\n The Decrypted text (in bytes): \n", d)

Enter: hello world!

 The Encrypted text (in bytes): 
 b'I\xb3`\xb7\xa6\xef\x10n\xe9\xb0\xf6^'

 The Decrypted text (in bytes): 
 b'hello world!'


#### - 3.2 DB 파일 암호화

In [40]:
import zlib # 압축을 위해 import

def crypt(inputFile):
    
    fp = open(inputFile, 'rb')
    fbuf = fp.read()
    fp.close()

    compressed = zlib.compress(fbuf)
    
    # ------------- 암호화된 DB 파일 생성 후 저장 ------------- #
    cypherText = AES_encrypt(key, fbuf) # 파일 암호화
    
    outputFile = inputFile.split(' ')[0] + '.secure' # 새로 만들어질 파일 이름

    fp = open(outputFile, 'wb')
    fp.write(cypherText)
    fp.close()
    # --------------------------------------------------------- #
    
    print(f'파일 암호화 완료!\n {inputFile} -> {outputFile}')
    

In [41]:
crypt('./malware.db')

파일 암호화 완료!
 ./malware.db -> ./malware.db.secure


#### - 3.3 암호화된 DB 파일 복호화하여 악성코드 정보 불러오기

In [42]:

# --- 복호화 ----------------------------------------------- #
def decrypt(inputFile):
    fp = open(inputFile, 'rb')
    fbuf = fp.read()
    fp.close()
    
    decrypted = AES_decrypt(key, fbuf)
    return decrypted


# --- 악성코드 정보 읽어오기 ------------------------------ #
def loadDB(fileName):
    fbuf = decrypt(fileName)
    patterns = fbuf.split()
    return patterns


# --- 악성코드 정보를 모아놓은 Database 생성 -------------- #
def makeDB(fileName):
    malwareDB = {}
    patterns = loadDB(fileName)
    
    for pattern in patterns: # 악성코드 정보 하나 당
        p = pattern.decode('utf-8').split(':')  # 콜론 기준으로 자르기
        
        name = p[0]                      # 악성코드 이름
        information = [p[1], int(p[2])]  # 그 해시값, 파일 크기
        malwareDB[name] = information    # DB에 추가해줌

    return malwareDB


#### - 3.4 악성코드를 진단하고 치료하는 백신 생성 (2.2와 동일)

In [43]:
def vaccine(fileLocation):
    malwareDB = makeDB('./malware.db.secure') # DB 가져오기
                                              # !! 이 부분만 다름
    sizeDB = map(lambda value: value[1], malwareDB.values()) # 악성코드 크기만

    fp = open(fileLocation, 'rb')
    
    # ------------ 속도 향상을 위한 파일 크기 색출 ------------ #
    fileSize = os.path.getsize(fileLocation)
    if fileSize not in sizeDB:
        print(f'-- Normal File \n{fileLocation}')
        return # 악성코드에 해당하는 파일크기가 아니라면 pass
    # --------------------------------------------------------- #
    
    fbuf = fp.read()
    fp.close()

    # ---------------------- 해시값 탐지 ---------------------- #
    f = hashlib.md5() # MD5 hash function
    f.update(fbuf)    # hashing
    hashValue = f.hexdigest() # 메시지 다이제스트 get (16진수 해시값)
    
    # 구한 해시값을 넣어 악성코드 여부와 그 이름 탐지
    isMalware, name = searchDB(hashValue, malwareDB) 
    # --------------------------------------------------------- #
    
    
    if isMalware == True:
        print(f'-- 악성코드 탐지( {name} ) \n{fileLocation}')
        os.chmod(fileLocation, 0o777)
        os.remove(fileLocation)
        print('-- 파일이 삭제되었습니다.\n')
    else: print(f'-- Normal File \n{fileLocation}')

In [44]:
vaccine('./malwares/eicar4.txt')
vaccine('./malwares/jihyo2.txt')
vaccine('./malwares/normal.txt')

-- 악성코드 탐지( EICAR ) 
./malwares/eicar4.txt
-- 파일이 삭제되었습니다.

-- 악성코드 탐지( JIHYO ) 
./malwares/jihyo2.txt
-- 파일이 삭제되었습니다.

-- Normal File 
./malwares/normal.txt


###### 참고 사이트: https://zong.tistory.com/13